In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)
from quara.data_analysis.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.data_analysis.weighted_relative_entropy import (
    WeightedRelativeEntropy,
    WeightedRelativeEntropyOption,
)
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import Povm, get_x_povm
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting
from quara.settings import Settings

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
start_all = time.time()

In [3]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_objects = [state_x0, state_y0, state_z0, state_z1]

In [4]:
# Case 1: 軸上
# a0, a1, a2, a3 = 1, 1, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 2: 軸から一番離れた表面
a0, a1, a2, a3 = 1, 1 / np.sqrt(2), 0, 1 / np.sqrt(2)
m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 3: 原点
# a0, a1, a2, a3 = 1, 0, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

In [8]:
num_data = [100, 1000, 10000]
n_rep = 10
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
    StandardPovmt(
        tester_objects,
        measurement_n,
        on_para_eq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardPovmt(
        tester_objects,
        measurement_n,
        on_para_eq_constraint=False,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardPovmt(
        tester_objects,
        measurement_n,
        on_para_eq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardPovmt(
        tester_objects,
        measurement_n,
        on_para_eq_constraint=False,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(4),
    WeightedRelativeEntropy(8),
    WeightedProbabilityBasedSquaredError(4),
    WeightedProbabilityBasedSquaredError(8),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption("identity"),
    WeightedRelativeEntropyOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)

    # stock settings of this simulation
    simulation_setting = StandardQTomographySimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        true_object=true_object,
        tester_objects=tester_objects,
    )
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 10%|█         | 1/10 [00:00<00:01,  7.83it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


 10%|█         | 1/10 [00:00<00:01,  8.00it/s]

elapsed_time:0.021472664674123128[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  0%|          | 0/10 [00:00<?, ?it/s]

...

elapsed_time:0.7372739315032959[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LossMinimizationEstimator


  0%|          | 0/10 [00:00<?, ?it/s]

elapsed_time:0.5928059856096903[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LossMinimizationEstimator


100%|██████████| 10/10 [00:28<00:00,  2.80s/it]

elapsed_time:0.4669113794962565[min]



In [9]:
report.export_report(f"povmt_nrep={n_rep}.pdf",
    estimation_results_list=estimation_results_list,   # 「EstimationResultのリスト」のリスト
    simulation_settings= simulation_settings,
    seed=seed,  # 推定で使ったseed（オプション）
    tolerance=Settings.get_atol()
)

​Generating table of computation time ...
​Generating table of tolerance of physicality constraint violation ...
​Generating table of experimental conditions ...
Generating case list ...
Computation time of estimators ...


100%|██████████| 10/10 [00:00<00:00, 10394.81it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  LinearEstimator(True)=[0.03203786692715932, 0.002837704178682182, 0.00024584712280149217]
  LinearEstimator(False)=[0.020277866927159317, 0.003541564504240555, 0.00032776280804385166]
  ProjectedLinearEstimator(True)=[0.026550064626018832, 0.0021114438702559877, 0.0002675468447344297]
  ProjectedLinearEstimator(False)=[0.015358630437802062, 0.002175824867230545, 0.0002449240055079414]
  Maximum-Likelihood(True)=[0.01877887048674481, 0.0012239607165757296, 0.0002339652521848516]
  Maximum-Likelihood(False)=[0.016421375940886357, 0.0028827906026858098, 0.00021025662704711917]
  Least Squares(True)=[0.016154841842252687, 0.0013013620418841603, 0.00014267036816194352]
  Least Squares(False)=[0.01540766028795649, 0.001834775646326154, 0.00021438008586254368]
  LinearEstimator(True)=[0.03203786692715932, 0.002837704178682182, 0.00024584712280149217]
  ProjectedLinearEstimator(True)=[0.026550064626018832, 0.00211144387

100%|██████████| 10/10 [00:00<00:00, 10716.16it/s]

​​Generating physicality violation test blocks ...



  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 38.82it/s]

2it [00:00, 36.39it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 26.08it/s]

2it [00:00, 30.24it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 49.30it/s]

2it [00:00, 55.70it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 42.33it/s]

2it [00:00, 42.04it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 21.30it/s]
 50%|█████     | 1/2 [00:00<00:00,  9.74it/s]
0it [00:00, ?it/s]
2it [00:00,  8.78it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 38.31it/s]

2it [00:00, 35.58it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 41.67it/s]

2it [00:00, 43.00it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 28.96it/s]

2it [00:00, 29.84it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 47.81it/s]

2it [00:00, 51.76it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 62.04it/s]

2it [00:00, 43.30it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 48.08it/s]

2it [00:00, 43.30it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (povmt_nrep=10.pdf)
